In [1]:
import os
import sys

sys.path.insert(0, os.getenv("HOME") + "/pycode/MscThesis/")
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    get_current_folders,
    get_folders_by_plate_id,
)

import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime, timedelta
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    sparse_to_doc,
)
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
)
from amftrack.pipeline.paths.directory import (
    run_parallel_transfer,
    find_state,
    directory_scratch,
    directory_project,
    directory_archive,
)
import dropbox
from amftrack.transfer.functions.transfer import upload, zip_file, sync_fold
from subprocess import call
from tqdm.autonotebook import tqdm
from time import time_ns
import subprocess

path_code = os.getenv("HOME") + "/pycode/MscThesis/"


def make_stitching_loop(directory, dirname, op_id):
    a_file = open(
        f"{path_code}amftrack/pipeline/scripts/stitching_loops/stitching_loop.ijm", "r"
    )

    list_of_lines = a_file.readlines()

    list_of_lines[4] = f"mainDirectory = \u0022{directory}\u0022 ;\n"
    list_of_lines[29] = f"\t if(startsWith(list[i],\u0022{dirname}\u0022)) \u007b\n"
    file_name = f'{os.getenv("TEMP")}/stitching_loops/stitching_loop{op_id}.ijm'
    a_file = open(file_name, "w")

    a_file.writelines(list_of_lines)

    a_file.close()


def run_stitch(directory, folders):
    folder_list = list(folders["folder"])
    folder_list.sort()
    with tqdm(total=len(folder_list), desc="stitched") as pbar:
        for folder in folder_list:
            op_id = time_ns()
            im = imageio.imread(f"{directory}/{folder}/Img/Img_r03_c05.tif")
            for x in range(1, 11):
                for y in range(1, 16):
                    strix = str(x) if x >= 10 else f"0{x}"
                    striy = str(y) if y >= 10 else f"0{y}"
                    path = f"{directory}/{folder}/Img/Img_r{strix}_c{striy}.tif"
                    # print(striy,path,os.path.getsize(path))
                    if not os.path.isfile(path):
                        f = open(path, "w")
                    if os.path.getsize(path) == 0:
                        imageio.imwrite(path, im * 0)
            make_stitching_loop(directory, folder, op_id)
            command = [
                os.getenv("HOME") + "/Fiji.app/ImageJ-linux64",
                "--mem=8000m",
                "--headless",
                "--ij2",
                "--console",
                "-macro",
                f'{os.getenv("TEMP")}/stitching_loops/stitching_loop{op_id}.ijm',
            ]
            print(" ".join(command))
            process = subprocess.run(command)
            pbar.update(1)


def run_parallel(code, args, folders):
    op_id = time_ns()
    folders.to_json(f'{os.getenv("TEMP")}/{op_id}.json')  # temporary file
    folder_list = list(folders["folder"])
    folder_list.sort()
    args_str = [str(arg) for arg in args]
    arg_str = " ".join(args_str)
    with tqdm(total=len(folder_list), desc="stitched") as pbar:
        for index, folder in enumerate(folder_list):
            command = (
                [
                    "python",
                    f"{path_code}amftrack/pipeline/scripts/image_processing/{code}",
                ]
                + args_str
                + [f"{op_id}", f"{index}"]
            )
            print(" ".join(command))
            process = subprocess.run(command)
            pbar.update(1)


# directory_origin = r'/mnt/sun/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE_syncing/'
directory_origin = r"/mnt/sun-temp/TEMP/PRINCE_syncing/"

# directory_target = r'/mnt/sun/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE_analysis/'
directory_target = r"/mnt/sun-temp/TEMP/PRINCE_analysis/"

update_plate_info(directory_origin)
update_plate_info(directory_target)

all_folders_origin = get_current_folders(directory_origin)
all_folders_target = get_current_folders(directory_target)
all_folders_origin["date_datetime"] = pd.to_datetime(
    all_folders_origin["date"].astype(str), format="%d.%m.%Y, %H:%M:"
)
selection = (datetime.now() - all_folders_origin["date_datetime"]) <= timedelta(
    hours=15
)
last_folders = all_folders_origin.loc[selection]
plates_to_stitch = last_folders["Plate"].unique()

/home/ipausers/bisot/pycode/MscThesis/amftrack/transfer/functions/transfer.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


analysed:   0%|          | 0/1476 [00:00<?, ?it/s]

analysed:   0%|          | 0/157 [00:00<?, ?it/s]

In [20]:
# all_folders_origin = get_current_folders(directory_origin)
# all_folders_target = get_current_folders(directory_target)
all_folders_origin["date_datetime"] = pd.to_datetime(
    all_folders_origin["date"].astype(str), format="%d.%m.%Y, %H:%M:"
)
selection = (datetime.now() - all_folders_origin["date_datetime"]) <= timedelta(
    hours=24
)
last_folders = all_folders_origin.loc[selection]
plates_to_stitch = last_folders["Plate"].unique()
len(plates_to_stitch)

10

In [23]:
len(last_folders["date_datetime"])

50

In [7]:
for plate in plates_to_stitch:
    folders = (
        all_folders_origin.loc[all_folders_origin["Plate"] == plate]
        .sort_values("date", ascending=False)
        .head(1)
    )
    print(folders["Plate"], folders["folder"])
    break
#     run_info = folders.copy()
#     target = directory_target
#     folder_list = list(run_info['total_path'])
#     for folder in folder_list:
#         origin = folder
#         sync_fold(origin,target)

# for plate in plates_to_stitch:
#     folders = all_folders_origin.loc[all_folders_origin['Plate']==plate].sort_values('date',ascending = False).head(1)
#     run_stitch(directory_target,folders)

2503    938
Name: Plate, dtype: object 2503    20220331_0344_Plate15
Name: folder, dtype: object


In [16]:
all_folders_origin["date"] = pd.to_datetime(
    all_folders_origin["date"], format="%d.%m.%Y, %H:%M:"
)

In [18]:
all_folders_origin.loc[all_folders_origin["Plate"] == plate].sort_values(
    "date", ascending=False
)["date"]

11508   2022-04-07 00:50:00
11525   2022-04-06 22:50:00
11521   2022-04-06 20:50:00
11543   2022-04-06 18:50:00
11552   2022-04-06 16:49:00
                ...        
1385    2022-03-26 05:42:00
1416    2022-03-26 03:42:00
1448    2022-03-26 01:42:00
1286    2022-03-25 23:42:00
1243    2022-03-25 21:42:00
Name: date, Length: 77, dtype: datetime64[ns]

In [ ]:
import os
import sys

sys.path.insert(0, os.getenv("HOME") + "/pycode/MscThesis/")
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    get_current_folders,
    get_folders_by_plate_id,
)

import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime, timedelta
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    sparse_to_doc,
)
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
)
from amftrack.pipeline.paths.directory import (
    run_parallel_transfer,
    find_state,
    directory_scratch,
    directory_project,
    directory_archive,
)
import dropbox
from amftrack.transfer.functions.transfer import upload, zip_file, sync_fold
from subprocess import call
from tqdm.autonotebook import tqdm
from time import time_ns
import subprocess

directory_origin = (
    r"/mnt/sun/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE_syncing/"
)
directory_origin = r"/mnt/sun-temp/TEMP/PRINCE_syncing/"

directory_target = (
    r"/mnt/sun/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE_analysis/"
)
directory_target = r"/mnt/sun-temp/TEMP/PRINCE_analysis/"

update_plate_info(directory_origin)
update_plate_info(directory_target)

all_folders_origin = get_current_folders(directory_origin)
all_folders_target = get_current_folders(directory_target)

In [ ]:
all_folders_origin["date_datetime"] = pd.to_datetime(
    all_folders_origin["date"].astype(str), format="%d.%m.%Y, %H:%M:"
)
selection = (datetime.now() - all_folders_origin["date_datetime"]) <= timedelta(hours=8)
last_folders = all_folders_origin.loc[selection]

In [ ]:
plates_to_stitch = last_folders["Plate"].unique()
# plates_target = all_folders_target['Plate'].unique()

In [ ]:
for plate in plates_to_stitch:
    folders = (
        all_folders_origin.loc[all_folders_origin["Plate"] == plate]
        .sort_values("date", ascending=False)
        .head(1)
    )
    run_info = folders.copy()
    target = directory_target
    folder_list = list(run_info["total_path"])
    for folder in folder_list:
        origin = folder
        sync_fold(origin, target)

sending incremental file list
20220315_0737_Plate27/
20220315_0737_Plate27/PrinceMeasurement.csv
20220315_0737_Plate27/param.m
20220315_0737_Plate27/Img/
20220315_0737_Plate27/Img/Img_r01_c05.tif
20220315_0737_Plate27/Img/Img_r01_c06.tif
20220315_0737_Plate27/Img/Img_r01_c07.tif
20220315_0737_Plate27/Img/Img_r01_c08.tif
20220315_0737_Plate27/Img/Img_r01_c09.tif
20220315_0737_Plate27/Img/Img_r01_c10.tif
20220315_0737_Plate27/Img/Img_r01_c11.tif
20220315_0737_Plate27/Img/Img_r02_c04.tif
20220315_0737_Plate27/Img/Img_r02_c05.tif
20220315_0737_Plate27/Img/Img_r02_c06.tif
20220315_0737_Plate27/Img/Img_r02_c07.tif
20220315_0737_Plate27/Img/Img_r02_c08.tif
20220315_0737_Plate27/Img/Img_r02_c09.tif
20220315_0737_Plate27/Img/Img_r02_c10.tif
20220315_0737_Plate27/Img/Img_r02_c11.tif
20220315_0737_Plate27/Img/Img_r02_c12.tif
20220315_0737_Plate27/Img/Img_r03_c03.tif
20220315_0737_Plate27/Img/Img_r03_c04.tif
20220315_0737_Plate27/Img/Img_r03_c05.tif
20220315_0737_Plate27/Img/Img_r03_c06.tif
202203

In [111]:
path_code = os.getenv("HOME") + "/pycode/MscThesis/"


def make_stitching_loop(directory, dirname, op_id):
    a_file = open(
        f"{path_code}amftrack/pipeline/scripts/stitching_loops/stitching_loop.ijm", "r"
    )

    list_of_lines = a_file.readlines()

    list_of_lines[4] = f"mainDirectory = \u0022{directory}\u0022 ;\n"
    list_of_lines[29] = f"\t if(startsWith(list[i],\u0022{dirname}\u0022)) \u007b\n"
    file_name = f'{os.getenv("TEMP")}/stitching_loops/stitching_loop{op_id}.ijm'
    a_file = open(file_name, "w")

    a_file.writelines(list_of_lines)

    a_file.close()


def run_stitch(directory, folders):
    folder_list = list(folders["folder"])
    folder_list.sort()
    with tqdm(total=len(folder_list), desc="stitched") as pbar:
        for folder in folder_list:
            op_id = time_ns()
            im = imageio.imread(f"{directory}/{folder}/Img/Img_r03_c05.tif")
            for x in range(1, 11):
                for y in range(1, 16):
                    strix = str(x) if x >= 10 else f"0{x}"
                    striy = str(y) if y >= 10 else f"0{y}"
                    path = f"{directory}/{folder}/Img/Img_r{strix}_c{striy}.tif"
                    # print(striy,path,os.path.getsize(path))
                    if not os.path.isfile(path):
                        f = open(path, "w")
                    if os.path.getsize(path) == 0:
                        imageio.imwrite(path, im * 0)
            make_stitching_loop(directory, folder, op_id)
            command = [
                os.getenv("HOME") + "/Fiji.app/ImageJ-linux64",
                "--mem=8000m",
                "--headless",
                "--ij2",
                "--console",
                "-macro",
                f'{os.getenv("TEMP")}/stitching_loops/stitching_loop{op_id}.ijm',
            ]
            print(" ".join(command))
            process = subprocess.run(command)
            pbar.update(1)


def run_parallel(code, args, folders):
    op_id = time_ns()
    folders.to_json(f'{os.getenv("TEMP")}/{op_id}.json')  # temporary file
    folder_list = list(folders["folder"])
    folder_list.sort()
    args_str = [str(arg) for arg in args]
    arg_str = " ".join(args_str)
    with tqdm(total=len(folder_list), desc="stitched") as pbar:
        for index, folder in enumerate(folder_list):
            command = (
                [
                    "python",
                    f"{path_code}amftrack/pipeline/scripts/image_processing/{code}",
                ]
                + args_str
                + [f"{op_id}", f"{index}"]
            )
            print(" ".join(command))
            process = subprocess.run(command)
            pbar.update(1)

In [112]:
for plate in plates_to_stitch:
    folders = (
        all_folders_origin.loc[all_folders_origin["Plate"] == plate]
        .sort_values("date", ascending=False)
        .head(1)
    )
    run_stitch(directory_target, folders)

stitched:   0%|          | 0/1 [00:00<?, ?it/s]

/home/ipausers/bisot/Fiji.app/ImageJ-linux64 --mem=8000m --headless --ij2 --console -macro /home/ipausers/bisot/temp//stitching_loops/stitching_loop1647354012846938265.ijm


Java HotSpot(TM) 64-Bit Server VM warning: ignoring option PermSize=128m; support was removed in 8.0
Java HotSpot(TM) 64-Bit Server VM warning: Using incremental CMS is deprecated and will likely be removed in a future release


20220315_0410_Plate39/
(Tue Mar 15 15:24:08 CET 2022): Stitching the following files:
Image: '/mnt/sun-temp/TEMP/PRINCE_analysis//20220315_0410_Plate39//Img2/Img_r01_c01.tif' Imp: 'null' Offset: (0.0, 0.0)
Image: '/mnt/sun-temp/TEMP/PRINCE_analysis//20220315_0410_Plate39//Img2/Img_r01_c02.tif' Imp: 'null' Offset: (3276.0, 0.0)
Image: '/mnt/sun-temp/TEMP/PRINCE_analysis//20220315_0410_Plate39//Img2/Img_r01_c03.tif' Imp: 'null' Offset: (6552.0, 0.0)
Image: '/mnt/sun-temp/TEMP/PRINCE_analysis//20220315_0410_Plate39//Img2/Img_r01_c04.tif' Imp: 'null' Offset: (9828.0, 0.0)
Image: '/mnt/sun-temp/TEMP/PRINCE_analysis//20220315_0410_Plate39//Img2/Img_r01_c05.tif' Imp: 'null' Offset: (13104.0, 0.0)
Image: '/mnt/sun-temp/TEMP/PRINCE_analysis//20220315_0410_Plate39//Img2/Img_r01_c06.tif' Imp: 'null' Offset: (16380.0, 0.0)
Image: '/mnt/sun-temp/TEMP/PRINCE_analysis//20220315_0410_Plate39//Img2/Img_r01_c07.tif' Imp: 'null' Offset: (19656.0, 0.0)
Image: '/mnt/sun-temp/TEMP/PRINCE_analysis//20220315_

stitched:   0%|          | 0/1 [00:00<?, ?it/s]

FileNotFoundError: No such file: '/mnt/sun-temp/TEMP/PRINCE_analysis/20220315_0622_Plate02/Img/Img_r03_c05.tif'

In [ ]:
directory_target

In [ ]:
1 + 1

In [ ]:
for plate in plates_to_stitch:
    folders = (
        all_folders_origin.loc[all_folders_origin["Plate"] == plate]
        .sort_values("date", ascending=False)
        .head(1)
    )
    run_info = folders.copy()
    target = directory_target
    folder_list = list(run_info["total_path"])
    for folder in folder_list:
        path = f"{directory_target}/{folder}/Img/"
        shutil.rmtree(path)